## Time Series Forecasting with the Long Short-Term Memory Network in Python

The Long Short-Term Memory recurrent neural network has the promise of learning long sequences of observations.

It seems a perfect match for time series forecasting, and in fact, it may be.

In this tutorial, you will discover how to develop an LSTM forecast model for a one-step univariate time series forecasting problem.

After completing this tutorial, you will know:

How to develop a baseline of performance for a forecast problem.
How to design a robust test harness for one-step time series forecasting.
How to prepare data, develop, and evaluate an LSTM recurrent neural network for time series forecasting.

### 1. Shampoo Sales Dataset

In [1]:
# load and plot dataset
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
series = read_csv('data/shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(series.head())
# line plot
series.plot()
pyplot.show()

Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales, dtype: float64


<Figure size 640x480 with 1 Axes>

### Complete Code


Using TensorFlow backend.


Epoch 1/1
 - 1s - loss: 0.2640
Epoch 1/1
 - 0s - loss: 0.2615
Epoch 1/1
 - 0s - loss: 0.2595
Epoch 1/1
 - 0s - loss: 0.2577
Epoch 1/1
 - 0s - loss: 0.2560
Epoch 1/1
 - 0s - loss: 0.2544
Epoch 1/1
 - 0s - loss: 0.2529
Epoch 1/1
 - 0s - loss: 0.2515
Epoch 1/1
 - 0s - loss: 0.2501
Epoch 1/1
 - 0s - loss: 0.2488
Epoch 1/1
 - 0s - loss: 0.2476
Epoch 1/1
 - 0s - loss: 0.2464
Epoch 1/1
 - 0s - loss: 0.2453
Epoch 1/1
 - 0s - loss: 0.2441
Epoch 1/1
 - 0s - loss: 0.2430
Epoch 1/1
 - 0s - loss: 0.2419
Epoch 1/1
 - 0s - loss: 0.2408
Epoch 1/1
 - 0s - loss: 0.2396
Epoch 1/1
 - 0s - loss: 0.2385
Epoch 1/1
 - 0s - loss: 0.2373
Epoch 1/1
 - 0s - loss: 0.2362
Epoch 1/1
 - 0s - loss: 0.2350
Epoch 1/1
 - 0s - loss: 0.2337
Epoch 1/1
 - 0s - loss: 0.2325
Epoch 1/1
 - 0s - loss: 0.2312
Epoch 1/1
 - 0s - loss: 0.2299
Epoch 1/1
 - 0s - loss: 0.2286
Epoch 1/1
 - 0s - loss: 0.2273
Epoch 1/1
 - 0s - loss: 0.2260
Epoch 1/1
 - 0s - loss: 0.2246
Epoch 1/1
 - 0s - loss: 0.2233
Epoch 1/1
 - 0s - loss: 0.2220
Epoch 1/

 - 0s - loss: 0.1049
Epoch 1/1
 - 0s - loss: 0.1048
Epoch 1/1
 - 0s - loss: 0.1047
Epoch 1/1
 - 0s - loss: 0.1046
Epoch 1/1
 - 0s - loss: 0.1044
Epoch 1/1
 - 0s - loss: 0.1043
Epoch 1/1
 - 0s - loss: 0.1042
Epoch 1/1
 - 0s - loss: 0.1041
Epoch 1/1
 - 0s - loss: 0.1039
Epoch 1/1
 - 0s - loss: 0.1038
Epoch 1/1
 - 0s - loss: 0.1037
Epoch 1/1
 - 0s - loss: 0.1035
Epoch 1/1
 - 0s - loss: 0.1034
Epoch 1/1
 - 0s - loss: 0.1033
Epoch 1/1
 - 0s - loss: 0.1032
Epoch 1/1
 - 0s - loss: 0.1030
Epoch 1/1
 - 0s - loss: 0.1029
Epoch 1/1
 - 0s - loss: 0.1028
Epoch 1/1
 - 0s - loss: 0.1027
Epoch 1/1
 - 0s - loss: 0.1025
Epoch 1/1
 - 0s - loss: 0.1024
Epoch 1/1
 - 0s - loss: 0.1023
Epoch 1/1
 - 0s - loss: 0.1022
Epoch 1/1
 - 0s - loss: 0.1021
Epoch 1/1
 - 0s - loss: 0.1019
Epoch 1/1
 - 0s - loss: 0.1018
Epoch 1/1
 - 0s - loss: 0.1017
Epoch 1/1
 - 0s - loss: 0.1016
Epoch 1/1
 - 0s - loss: 0.1015
Epoch 1/1
 - 0s - loss: 0.1013
Epoch 1/1
 - 0s - loss: 0.1012
Epoch 1/1
 - 0s - loss: 0.1011
Epoch 1/1
 - 0s - 

Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0920
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/1
 - 0s - loss: 0.0919
Epoch 1/

 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0897
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0895
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0895
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - loss: 0.0895
Epoch 1/1
 - 0s - loss: 0.0896
Epoch 1/1
 - 0s - 

Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0880
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0877
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0877
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0877
Epoch 1/

 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0873
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0870
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0870
Epoch 1/1
 - 0s - loss: 0.0870
Epoch 1/1
 - 0s - loss: 0.0870
Epoch 1/1
 - 0s - loss: 0.0874
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0873
Epoch 1/1
 - 0s - loss: 0.0868
Epoch 1/1
 - 0s - loss: 0.0875
Epoch 1/1
 - 0s - loss: 0.0868
Epoch 1/1
 - 0s - loss: 0.0877
Epoch 1/1
 - 0s - loss: 0.0867
Epoch 1/1
 - 0s - loss: 0.0881
Epoch 1/1
 - 0s - loss: 0.0868
Epoch 1/1
 - 0s - loss: 0.0880
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0883
Epoch 1/1
 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - 

Epoch 1/1
 - 0s - loss: 0.0875
Epoch 1/1
 - 0s - loss: 0.0858
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0862
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0862
Epoch 1/1
 - 0s - loss: 0.0875
Epoch 1/1
 - 0s - loss: 0.0858
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0863
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0862
Epoch 1/1
 - 0s - loss: 0.0878
Epoch 1/1
 - 0s - loss: 0.0855
Epoch 1/1
 - 0s - loss: 0.0875
Epoch 1/1
 - 0s - loss: 0.0860
Epoch 1/1
 - 0s - loss: 0.0887
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0874
Epoch 1/1
 - 0s - loss: 0.0857
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0857
Epoch 1/1
 - 0s - loss: 0.0876
Epoch 1/1
 - 0s - loss: 0.0861
Epoch 1/1
 - 0s - loss: 0.0880
Epoch 1/1
 - 0s - loss: 0.0864
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0862
Epoch 1/1
 - 0s - loss: 0.0872
Epoch 1/1
 - 0s - loss: 0.0856
Epoch 1/1
 - 0s - loss: 0.0875
Epoch 1/1
 - 0s - loss: 0.0860
Epoch 1/

 - 0s - loss: 0.0869
Epoch 1/1
 - 0s - loss: 0.0879
Epoch 1/1
 - 0s - loss: 0.0855
Epoch 1/1
 - 0s - loss: 0.0852
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0845
Epoch 1/1
 - 0s - loss: 0.0850
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0860
Epoch 1/1
 - 0s - loss: 0.0851
Epoch 1/1
 - 0s - loss: 0.0868
Epoch 1/1
 - 0s - loss: 0.0881
Epoch 1/1
 - 0s - loss: 0.0877
Epoch 1/1
 - 0s - loss: 0.0892
Epoch 1/1
 - 0s - loss: 0.0852
Epoch 1/1
 - 0s - loss: 0.0845
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0846
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0852
Epoch 1/1
 - 0s - loss: 0.0851
Epoch 1/1
 - 0s - loss: 0.0871
Epoch 1/1
 - 0s - loss: 0.0893
Epoch 1/1
 - 0s - loss: 0.0873
Epoch 1/1
 - 0s - loss: 0.0882
Epoch 1/1
 - 0s - loss: 0.0848
Epoch 1/1
 - 0s - loss: 0.0843
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0845
Epoch 1/1
 - 0s - loss: 0.0843
Epoch 1/1
 - 0s - 

Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0843
Epoch 1/

 - 0s - loss: 0.0848
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/1
 - 0s - loss: 0.0847
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/1
 - 0s - loss: 0.0846
Epoch 1/1
 - 0s - loss: 0.0836
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0856
Epoch 1/1
 - 0s - loss: 0.0846
Epoch 1/1
 - 0s - loss: 0.0861
Epoch 1/1
 - 0s - loss: 0.0851
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0835
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0859
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0854
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0848
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/1
 - 0s - loss: 0.0849
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0852
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0853
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0851
Epoch 1/1
 - 0s - 

Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0836
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0836
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0836
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0837
Epoch 1/

 - 0s - loss: 0.0900
Epoch 1/1
 - 0s - loss: 0.0887
Epoch 1/1
 - 0s - loss: 0.0876
Epoch 1/1
 - 0s - loss: 0.0868
Epoch 1/1
 - 0s - loss: 0.0862
Epoch 1/1
 - 0s - loss: 0.0857
Epoch 1/1
 - 0s - loss: 0.0853
Epoch 1/1
 - 0s - loss: 0.0850
Epoch 1/1
 - 0s - loss: 0.0848
Epoch 1/1
 - 0s - loss: 0.0846
Epoch 1/1
 - 0s - loss: 0.0845
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0844
Epoch 1/1
 - 0s - loss: 0.0843
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0842
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0841
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0840
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - loss: 0.0839
Epoch 1/1
 - 0s - 

In [2]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

# load dataset
series = read_csv('data/shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# fit the model
lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = raw_values[len(train) + i + 1]
	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(raw_values[-12:])
pyplot.plot(predictions)
pyplot.show()